In [ ]:
import os

import numpy as np
import pandas as pd

from covidvu import utils
from covidvu.config import MASTER_DATABASE
from covidvu.cryostation import Cryostation
from tqdm.auto import tqdm

In [ ]:
WINDOW_SIZE = 7
MIN_CASES = 100

In [ ]:
databasePath = MASTER_DATABASE
with Cryostation(databasePath) as cryostation:
    countries = cryostation.timeSeriesFor(regionType = 'country', casesType = 'confirmed', disableProgressBar=False)

In [ ]:
with Cryostation(databasePath) as cryostation:
    statesUS = cryostation.timeSeriesFor(regionType = 'province', countryName='US', 
                                     casesType = 'confirmed', disableProgressBar=False)

In [ ]:
def computeGrowthFactor(confirmedCases, windowSize):    
    confirmedCasesSmooth = confirmedCases.copy().rolling(windowSize, axis=0).mean()
    newCases = confirmedCasesSmooth.diff(axis=0)
    growthFactor = newCases/newCases.shift(1)
    return growthFactor

In [ ]:
growthCountries = computeGrowthFactor(countries, WINDOW_SIZE)

In [ ]:
growthStatesUS = computeGrowthFactor(statesUS, WINDOW_SIZE)

In [ ]:
todayDate = pd.Timestamp.today().date()
yesterdayDate = (pd.Timestamp.today() - pd.Timedelta(1,'D')).date()
lastWeekDate = (pd.Timestamp.today() - pd.Timedelta(7,'D')).date()
lastTwoWeekDate = (pd.Timestamp.today() - pd.Timedelta(14,'D')).date()

In [ ]:
def getGrowthGaugeData(growth):
    yesterdayGrowth = growth.loc[yesterdayDate]
    lastWeekGrowth = growth.loc[lastWeekDate]
    lastTwoWeekGrowth = growth.loc[lastTwoWeekDate]
    return {
                'yesterday': yesterdayGrowth, 
                'lastWeek': lastWeekGrowth, 
                'lastTwoWeek': lastTwoWeekGrowth,
           }

In [ ]:
growthCountries.index = growthCountries.index.map(lambda t: t.date())

growthStatesUS.index = growthStatesUS.index.map(lambda t: t.date())

growthGaugeDataCountries = getGrowthGaugeData(growthCountries)
growthGaugeDataStatesUS  = getGrowthGaugeData(growthStatesUS)

In [ ]:
growthCountries.columns

In [ ]:
disableProgressBar=False

In [ ]:
for countryName in tqdm(growthCountries.columns, disable=disableProgressBar):
    with Cryostation(databasePath) as cryostation:
        country = cryostation[countryName]
        country['growth'] = {d:growthGaugeDataCountries[d][countryName] for d in growthGaugeDataCountries.keys()}
        cryostation[countryName] = country

In [ ]:
for stateName in tqdm(statesUS.columns, disable=disableProgressBar):
    with Cryostation(databasePath) as cryostation:
        country = cryostation['US']
        province = country['provinces'][stateName]
        province['growth'] = {d:growthGaugeDataStatesUS[d][stateName] for d in growthGaugeDataStatesUS.keys()}
        country['provinces'][stateName] = province
        cryostation['US'] = country

In [ ]:
with Cryostation(databasePath) as cryostation:
        country = cryostation['US']
        province = country['provinces']['Alabama']
        growth = province['growth']
growth